In [1]:
from confluent_kafka import Consumer, KafkaError
from confluent_kafka.schema_registry import SchemaRegistryClient
from confluent_kafka.schema_registry.protobuf import ProtobufDeserializer
from confluent_kafka.serialization import SerializationContext, MessageField
import my_event_pb2

In [2]:
def is_prime(n):
    if n < 2:
        return False
    for i in range(2, int(n ** 0.5) + 1):
        if n % i == 0:
            return False
    return True

In [3]:
schema_registry_conf = {'url': 'http://schema-registry:8081'}
schema_registry_client = SchemaRegistryClient(schema_registry_conf)

deserializer_conf = {'use.deprecated.format': False}
protobuf_deserializer = ProtobufDeserializer(my_event_pb2.MyEvent, deserializer_conf)

/opt/conda/lib/python3.11/site-packages/confluent_kafka/schema_registry/protobuf.py:513: UserWarning: MessageFactory class is deprecated. Please use GetMessageClass() instead of MessageFactory.GetPrototype. MessageFactory class will be removed after 2024.
  self._msg_class = MessageFactory().GetPrototype(descriptor)


In [4]:
c = Consumer({
    'bootstrap.servers': 'kafka-1:29092,kafka-2:29093,kafka-3:29094',
    'group.id': 'mygroup',
    'auto.offset.reset': 'earliest'
})

c.subscribe(['my-topic'])

while True:
    msg = c.poll(5.0)
    if msg is None:
        continue
    if msg.error():
        if msg.error().code() == KafkaError._PARTITION_EOF:
            continue
        else:
            print(msg.error())
            break

    event = protobuf_deserializer(msg.value(), SerializationContext(msg.topic(), MessageField.VALUE))
    
    original_value = event.value
    is_prime_number = is_prime(original_value)

    print(f'Received event with timestamp: {event.timestamp}')
    print(f'Prime Checker Consumer - Original: {original_value}, Is it Prime?: {is_prime_number}\n')

c.close()

Received event with timestamp: seconds: 1725445736
nanos: 143423000

Prime Checker Consumer - Original: 36, Is it Prime?: False

Received event with timestamp: seconds: 1725445741
nanos: 145617000

Prime Checker Consumer - Original: 22, Is it Prime?: False

Received event with timestamp: seconds: 1725445766
nanos: 154364000

Prime Checker Consumer - Original: 90, Is it Prime?: False



KeyboardInterrupt: 